In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Unzipping the zipped folder to extract files inside
!unzip /content/drive/MyDrive/food45.zip

Archive:  /content/drive/MyDrive/food45.zip
replace images/apple_pie/1005649.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [2]:
image_dir = Path('/content/images')
image_dir

PosixPath('/content/images')

In [3]:
file_paths = list(image_dir.glob(r'**/*.jpg'))

In [4]:
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1],file_paths))
labels

['chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_cake',
 'chocolate_

In [5]:
file_paths = pd.Series(file_paths, name='Filepath').astype(str)
labels = pd.Series(labels,name='Label')
images = pd.concat([file_paths,labels],axis=1)
images

,Filepath,Label
0,/content/images/chocolate_cake/2299970.jpg,chocolate_cake
1,/content/images/chocolate_cake/1760905.jpg,chocolate_cake
2,/content/images/chocolate_cake/1571592.jpg,chocolate_cake
3,/content/images/chocolate_cake/3102746.jpg,chocolate_cake
4,/content/images/chocolate_cake/3842697.jpg,chocolate_cake
...,...,...
44995,/content/images/hamburger/1626383.jpg,hamburger
44996,/content/images/hamburger/3452947.jpg,hamburger
44997,/content/images/hamburger/3543840.jpg,hamburger
44998,/content/images/hamburger/1823305.jpg,hamburger


In [6]:
images['Label'].value_counts()

chocolate_cake             1000
breakfast_burrito          1000
strawberry_shortcake       1000
pancakes                   1000
garlic_bread               1000
chicken_wings              1000
caprese_salad              1000
churros                    1000
apple_pie                  1000
hot_and_sour_soup          1000
french_toast               1000
cup_cakes                  1000
cheesecake                 1000
samosa                     1000
macaroni_and_cheese        1000
onion_rings                1000
pizza                      1000
bruschetta                 1000
chicken_curry              1000
carrot_cake                1000
miso_soup                  1000
waffles                    1000
dumplings                  1000
macarons                   1000
tacos                      1000
omelette                   1000
caesar_salad               1000
spaghetti_bolognese        1000
sushi                      1000
falafel                    1000
french_fries               1000
red_velv

In [7]:
train_df, test_df = train_test_split(images, train_size=0.7, shuffle=True, random_state=1)

In [8]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

In [9]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)

Found 25200 validated image filenames belonging to 45 classes.
Found 6299 validated image filenames belonging to 45 classes.
Found 13501 validated image filenames belonging to 45 classes.


In [10]:
pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    weights='imagenet',
    pooling='avg'
)

pretrained_model.trainable = False

In [11]:
inputs = pretrained_model.input

x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)

outputs = tf.keras.layers.Dense(45, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)


print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [13]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=30,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

Epoch 1/30
788/788 [==============================] - 197s 243ms/step - loss: 2.7606 - accuracy: 0.2851 - val_loss: 2.4161 - val_accuracy: 0.3404
Epoch 2/30
788/788 [==============================] - 172s 218ms/step - loss: 2.2826 - accuracy: 0.3736 - val_loss: 2.2776 - val_accuracy: 0.3780
Epoch 3/30
788/788 [==============================] - 179s 227ms/step - loss: 2.1562 - accuracy: 0.4008 - val_loss: 2.1954 - val_accuracy: 0.3924
Epoch 4/30
788/788 [==============================] - 173s 219ms/step - loss: 2.0636 - accuracy: 0.4210 - val_loss: 2.1408 - val_accuracy: 0.4043
Epoch 5/30
788/788 [==============================] - 173s 220ms/step - loss: 1.9876 - accuracy: 0.4401 - val_loss: 2.0954 - val_accuracy: 0.4159
Epoch 6/30
788/788 [==============================] - 175s 221ms/step - loss: 1.9235 - accuracy: 0.4568 - val_loss: 2.0531 - val_accuracy: 0.4398
Epoch 7/30
788/788 [==============================] - 175s 222ms/step - loss: 1.8647 - accuracy: 0.4667 - val_loss: 2.0299 -

In [14]:
results = model.evaluate(test_images, verbose=0)
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

Test Accuracy: 51.63%


In [15]:
predictions = np.argmax(model.predict(test_images), axis=1)
clr = classification_report(test_images.labels, predictions, target_names=test_images.class_indices, zero_division=0)

422/422 [==============================] - 73s 171ms/step


In [16]:
print("Classification Report:\n----------------------\n", clr)

Classification Report:
----------------------
                          precision    recall  f1-score   support

              apple_pie       0.33      0.21      0.26       308
      breakfast_burrito       0.45      0.62      0.52       277
             bruschetta       0.39      0.33      0.36       288
           caesar_salad       0.55      0.52      0.54       285
          caprese_salad       0.37      0.51      0.43       302
            carrot_cake       0.42      0.33      0.37       307
             cheesecake       0.35      0.23      0.28       291
          chicken_curry       0.38      0.38      0.38       319
          chicken_wings       0.49      0.71      0.58       292
         chocolate_cake       0.38      0.52      0.44       310
                churros       0.58      0.56      0.57       299
          club_sandwich       0.52      0.51      0.52       286
              cup_cakes       0.54      0.59      0.56       282
                 donuts       0.54      0.